In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [1]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import DOMINANT, OCGNN, CONAD, GAE, AnomalyDAE
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
import pickle
import time

## Graph

### Load Graph Skenario  1

In [8]:
train_graph = pickle.load(open('model_graph/train_graph.pkl', 'rb'))
label_train = pickle.load(open('model_graph/label_train.pkl', 'rb'))
train_node_features = pickle.load(open('model_graph/train_node_features.pkl', 'rb'))

In [9]:
test_graph = pickle.load(open('model_graph/test_graph.pkl', 'rb'))
label_test = pickle.load(open('model_graph/label_test.pkl', 'rb'))
test_node_features = pickle.load(open('model_graph/test_node_features.pkl', 'rb'))

In [10]:
## number of nodes
print(train_graph.number_of_nodes())
print(test_graph.number_of_nodes())

15451
9525


In [11]:
# print num of isolated nodes
print(len(list(nx.isolates(train_graph))))
# print num of isolated nodes
print(len(list(nx.isolates(test_graph))))

0
0


In [12]:
#convert label_train to numpy array
label_train_np = np.array(label_train)
label_train_torch = torch.from_numpy(label_train_np)
unique, counts = torch.unique(label_train_torch, return_counts=True)
dict(zip(unique, counts))

{tensor(0): tensor(10070), tensor(1): tensor(5381)}

### Load Graph Skenario 2

In [13]:
train_graph_2 = pickle.load(open('model_graph/train_graph_port.pkl', 'rb'))
label_train_2 = pickle.load(open('model_graph/label_train_port.pkl', 'rb'))
train_node_features_2 = pickle.load(open('model_graph/train_node_features_ip_port.pkl', 'rb'))

In [14]:
test_graph_2 = pickle.load(open('model_graph/test_graph_port.pkl', 'rb'))
label_test_2 = pickle.load(open('model_graph/label_test_port.pkl', 'rb'))
test_node_features_2 = pickle.load(open('model_graph/test_node_features_port.pkl', 'rb'))

In [15]:
# print num of isolated nodes
print(len(list(nx.isolates(train_graph_2))))
# print num of isolated nodes
print(len(list(nx.isolates(test_graph_2))))

11213
6999


In [16]:
## number of nodes
print(train_graph_2.number_of_nodes())
print(test_graph_2.number_of_nodes())

15480
8604


In [17]:
print(len(label_train_2))
print(len(label_test_2))

15480
8604


### Load Graph Skenario 3

In [18]:
train_graph_3 = pickle.load(open('model_graph/train_graph_ip_port.pkl', 'rb'))
label_train_3 = pickle.load(open('model_graph/label_train_ip_port.pkl', 'rb'))
train_node_features_3 = pickle.load(open('model_graph/train_node_features_ip_port.pkl', 'rb'))

In [19]:
test_graph_3 = pickle.load(open('model_graph/test_graph_ip_port.pkl', 'rb'))
label_test_3 = pickle.load(open('model_graph/label_test_ip_port.pkl', 'rb'))
test_node_features_3 = pickle.load(open('model_graph/test_node_features_ip_port.pkl', 'rb'))

In [20]:
# print num of isolated nodes
print(len(list(nx.isolates(train_graph_3))))
# print num of isolated nodes
print(len(list(nx.isolates(test_graph_3))))

13629
7832


In [21]:
## number of nodes
print(train_graph_3.number_of_nodes())
print(test_graph_3.number_of_nodes())

15480
8604


In [22]:
print(len(train_node_features_3))
print(len(test_node_features_3))

15480
8604


In [23]:
print(len(label_train_3))
print(len(label_test_3))

15480
8604


## Model with default parameter

### DOMINANT

In [24]:
def predict_dominant(label_test, dominant_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features
    print(pyG_test.x.shape)
    dominant_ip_pred_res, dominant_ip_score_res = dominant_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True)
    
    precision_pygod = eval_precision_at_k(label_test, dominant_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, dominant_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return precision_pygod, recall_pygod, f1_score

def make_dominant_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    print(pyG_train.x.shape)
    dominant_model = DOMINANT(epoch=5, verbose=3, gpu=-1)
    dominant_compile = dominant_model.fit(pyG_train)
    return dominant_compile

#### skenario 1

In [74]:
dominant_model = make_dominant_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_dominant(label_test, dominant_model, test_graph, test_node_features)

torch.Size([15457, 1])
Epoch 0000: Loss 394326.3438 |  | Time 4.12
Epoch 0001: Loss 158415.8594 |  | Time 3.46
Epoch 0002: Loss 57213.7812 |  | Time 3.83
Epoch 0003: Loss 24591.7090 |  | Time 3.46
Epoch 0004: Loss 20910.1738 |  | Time 3.70
torch.Size([6955, 1])
Test: Loss 3.8285 | AUC 0.8984 | Recall 0.7909 | Precision 0.7909 | AP 0.8871 | F1 0.7909 | Time 0.43
F1 score:  tensor(0.7909)
Precision:  tensor(0.7909)
Recall:  tensor(0.7909)


#### skenario 2

In [25]:
dominant_model = make_dominant_model(train_graph_2, train_node_features_2, label_train_2)
precision_score, recall_score, f1_score = predict_dominant(label_test_2, dominant_model, test_graph_2, test_node_features_2)

torch.Size([15480, 1])


IndexError: index 15473 is out of bounds for dimension 0 with size 15473

#### skenario 3

In [45]:
dominant_model = make_dominant_model(train_graph_3, train_node_features_3, label_train_3)
precision_score, recall_score, f1_score = predict_dominant(label_test_3, dominant_model, test_graph_3, test_node_features_3)

torch.Size([15480, 1])


IndexError: index 15473 is out of bounds for dimension 0 with size 15473

### OCGNN

In [31]:
def predict_ocgnn(label_test, ocgnn_compile, pyG_test, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(pyG_test)
    pyG_test.x = test_node_features

    ocgnn_ip_pred_res, ocgnn_ip_score_res = ocgnn_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')

    precision_pygod = eval_precision_at_k(label_test, ocgnn_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, ocgnn_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    f1_pygod = eval_f1(label_test, ocgnn_ip_pred_res)
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    print("F1 pygod: ", f1_pygod)

    return precision_pygod, recall_pygod, f1_score

def make_ocgnn_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features

    ocgnn_model = OCGNN(epoch=100, verbose=3)
    ocgnn_compile = ocgnn_model.fit(pyG_train)
    return ocgnn_compile

#### Skenario 1

In [82]:
ocgnn_model = make_ocgnn_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_ocgnn(label_test, ocgnn_model, test_graph, test_node_features)

Epoch 0000: Loss 38821.9766 |  | Time 0.10
Epoch 0001: Loss 23365.4883 |  | Time 0.17
Epoch 0002: Loss 16890.0449 |  | Time 0.09
Epoch 0003: Loss 12041.4941 |  | Time 0.09
Epoch 0004: Loss 8644.4150 |  | Time 0.10
Epoch 0005: Loss 6417.8452 |  | Time 0.10
Epoch 0006: Loss 5052.6890 |  | Time 0.10
Epoch 0007: Loss 4352.5947 |  | Time 0.08
Epoch 0008: Loss 4159.5566 |  | Time 0.09
Epoch 0009: Loss 4206.7051 |  | Time 0.11
Epoch 0010: Loss 4318.9199 |  | Time 0.12
Epoch 0011: Loss 4421.4116 |  | Time 0.11
Epoch 0012: Loss 4475.7905 |  | Time 0.10
Epoch 0013: Loss 4466.1040 |  | Time 0.09
Epoch 0014: Loss 4390.8037 |  | Time 0.10
Epoch 0015: Loss 4259.7285 |  | Time 0.10
Epoch 0016: Loss 4090.1499 |  | Time 0.10
Epoch 0017: Loss 3900.6592 |  | Time 0.11
Epoch 0018: Loss 3706.2373 |  | Time 0.09
Epoch 0019: Loss 3517.1367 |  | Time 0.10
Epoch 0020: Loss 3340.6465 |  | Time 0.10
Epoch 0021: Loss 3182.5569 |  | Time 0.12
Epoch 0022: Loss 3051.9395 |  | Time 0.09
Epoch 0023: Loss 2963.8274 |  

#### skenario 2

In [32]:
ocgnn_model = make_ocgnn_model(train_graph_2, train_node_features_2, label_train_2)
precision_score, recall_score, f1_score = predict_ocgnn(label_test_2, ocgnn_model, test_graph_2, test_node_features_2)

Epoch 0000: Loss 272329.4688 |  | Time 0.22
Epoch 0001: Loss 196284.0312 |  | Time 0.07
Epoch 0002: Loss 146980.3125 |  | Time 0.05
Epoch 0003: Loss 109201.5938 |  | Time 0.06
Epoch 0004: Loss 80649.5156 |  | Time 0.06
Epoch 0005: Loss 59695.8398 |  | Time 0.08
Epoch 0006: Loss 44741.0273 |  | Time 0.08
Epoch 0007: Loss 34311.1680 |  | Time 0.06
Epoch 0008: Loss 27367.6602 |  | Time 0.06
Epoch 0009: Loss 23044.4102 |  | Time 0.05
Epoch 0010: Loss 20421.2363 |  | Time 0.06
Epoch 0011: Loss 18928.2773 |  | Time 0.07
Epoch 0012: Loss 18156.6289 |  | Time 0.06
Epoch 0013: Loss 17805.0645 |  | Time 0.06
Epoch 0014: Loss 17627.0273 |  | Time 0.06
Epoch 0015: Loss 17443.8516 |  | Time 0.06
Epoch 0016: Loss 17160.8320 |  | Time 0.12
Epoch 0017: Loss 16752.4473 |  | Time 0.06
Epoch 0018: Loss 16233.4678 |  | Time 0.11
Epoch 0019: Loss 15633.1641 |  | Time 0.05
Epoch 0020: Loss 14983.0029 |  | Time 0.06
Epoch 0021: Loss 14312.9004 |  | Time 0.07
Epoch 0022: Loss 13647.0371 |  | Time 0.06
Epoch 0

#### skenario 3

In [84]:
ocgnn_model = make_ocgnn_model(train_graph_3, train_node_features_3, label_train_3)
precision_score, recall_score, f1_score = predict_ocgnn(label_test_3, ocgnn_model, test_graph_3, test_node_features_3)

Epoch 0000: Loss 145493.6875 |  | Time 0.06
Epoch 0001: Loss 98338.7734 |  | Time 0.06
Epoch 0002: Loss 68696.9219 |  | Time 0.06
Epoch 0003: Loss 47724.0703 |  | Time 0.06
Epoch 0004: Loss 33233.4102 |  | Time 0.07
Epoch 0005: Loss 23495.0137 |  | Time 0.08
Epoch 0006: Loss 17337.1211 |  | Time 0.06
Epoch 0007: Loss 13717.0420 |  | Time 0.06
Epoch 0008: Loss 11834.3447 |  | Time 0.06
Epoch 0009: Loss 10928.4941 |  | Time 0.05
Epoch 0010: Loss 10522.3604 |  | Time 0.05
Epoch 0011: Loss 10321.0176 |  | Time 0.06
Epoch 0012: Loss 10160.2988 |  | Time 0.08
Epoch 0013: Loss 9954.0576 |  | Time 0.09
Epoch 0014: Loss 9664.7900 |  | Time 0.07
Epoch 0015: Loss 9292.3750 |  | Time 0.08
Epoch 0016: Loss 8863.8682 |  | Time 0.07
Epoch 0017: Loss 8417.5449 |  | Time 0.08
Epoch 0018: Loss 7987.0801 |  | Time 0.06
Epoch 0019: Loss 7592.3994 |  | Time 0.06
Epoch 0020: Loss 7238.5469 |  | Time 0.06
Epoch 0021: Loss 6919.6494 |  | Time 0.06
Epoch 0022: Loss 6623.9551 |  | Time 0.07
Epoch 0023: Loss 633

### GAE

In [14]:
def predict_gae(label_test, gae_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    gae_ip_pred_res, gae_ip_score_res = gae_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    precision = eval_precision_at_k(label_test, gae_ip_score_res)
    recall = eval_recall_at_k(label_test, gae_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return precision, recall, f1_score
    
def make_gae_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train

    gae_model = GAE(epoch=100, verbose=3)
    gae_compile = gae_model.fit(pyG_train, label_train)
    return gae_compile

#### Skenario 1

In [15]:
gae_model = make_gae_model(train_graph_2, train_node_features_2, label_train_2)
precision_score, recall_score, f1_score = predict_gae(label_test_2, gae_model, test_graph_2, test_node_features_2)

Epoch 0000: Loss 141640.9844 | AUC 0.7838 | Recall 0.6622 | Precision 0.6622 | AP 0.6506 | F1 0.7159 | Time 0.15
Epoch 0001: Loss 86944.0312 | AUC 0.7773 | Recall 0.6523 | Precision 0.6523 | AP 0.6435 | F1 0.6720 | Time 0.08
Epoch 0002: Loss 50552.5000 | AUC 0.7648 | Recall 0.6562 | Precision 0.6562 | AP 0.6256 | F1 0.6684 | Time 0.11
Epoch 0003: Loss 27243.8516 | AUC 0.7514 | Recall 0.6709 | Precision 0.6709 | AP 0.5953 | F1 0.6704 | Time 0.09
Epoch 0004: Loss 17069.3672 | AUC 0.6877 | Recall 0.5117 | Precision 0.5117 | AP 0.5125 | F1 0.5106 | Time 0.11
Epoch 0005: Loss 19053.2266 | AUC 0.6940 | Recall 0.6158 | Precision 0.6158 | AP 0.5277 | F1 0.6157 | Time 0.11
Epoch 0006: Loss 25842.7031 | AUC 0.6901 | Recall 0.6182 | Precision 0.6182 | AP 0.5349 | F1 0.6182 | Time 0.09
Epoch 0007: Loss 28520.9570 | AUC 0.6882 | Recall 0.6130 | Precision 0.6130 | AP 0.5356 | F1 0.6149 | Time 0.09
Epoch 0008: Loss 26046.2812 | AUC 0.6903 | Recall 0.6179 | Precision 0.6179 | AP 0.5350 | F1 0.6180 | T

#### skenario 2

In [25]:
gae_model = make_gae_model(train_graph_2, train_node_features_2, label_train_2)
precision_score, recall_score, f1_score = predict_gae(label_test_2, gae_model, test_graph_2, test_node_features_2)

Epoch 0000: Loss 102309.2344 | AUC 0.7763 | Recall 0.6492 | Precision 0.6492 | AP 0.6447 | F1 0.6690 | Time 0.08
Epoch 0001: Loss 67907.7891 | AUC 0.7665 | Recall 0.6516 | Precision 0.6516 | AP 0.6314 | F1 0.6686 | Time 0.10
Epoch 0002: Loss 45399.6367 | AUC 0.7588 | Recall 0.6574 | Precision 0.6574 | AP 0.6162 | F1 0.6651 | Time 0.09
Epoch 0003: Loss 30760.7305 | AUC 0.7462 | Recall 0.6618 | Precision 0.6618 | AP 0.5940 | F1 0.6617 | Time 0.09
Epoch 0004: Loss 21408.2969 | AUC 0.7249 | Recall 0.6318 | Precision 0.6318 | AP 0.5648 | F1 0.6305 | Time 0.09
Epoch 0005: Loss 19268.4219 | AUC 0.6746 | Recall 0.5032 | Precision 0.5032 | AP 0.4991 | F1 0.5025 | Time 0.10
Epoch 0006: Loss 23888.2012 | AUC 0.6934 | Recall 0.6150 | Precision 0.6150 | AP 0.5351 | F1 0.6085 | Time 0.10
Epoch 0007: Loss 29230.3398 | AUC 0.6928 | Recall 0.6176 | Precision 0.6176 | AP 0.5395 | F1 0.6176 | Time 0.10
Epoch 0008: Loss 29762.4082 | AUC 0.6924 | Recall 0.6174 | Precision 0.6174 | AP 0.5395 | F1 0.6174 | T

#### skenario 3

In [24]:
gae_model = make_gae_model(train_graph_3, train_node_features_3, label_train_3)
precision_score, recall_score, f1_score = predict_gae(label_test_3, gae_model, test_graph_3, test_node_features_3)

Epoch 0000: Loss 75579.4141 | AUC 0.7804 | Recall 0.6536 | Precision 0.6536 | AP 0.6499 | F1 0.6940 | Time 0.08
Epoch 0001: Loss 36112.0898 | AUC 0.7673 | Recall 0.6503 | Precision 0.6503 | AP 0.6330 | F1 0.6907 | Time 0.08
Epoch 0002: Loss 14729.1211 | AUC 0.7607 | Recall 0.6523 | Precision 0.6523 | AP 0.6214 | F1 0.6919 | Time 0.09
Epoch 0003: Loss 8175.8115 | AUC 0.7263 | Recall 0.6770 | Precision 0.6770 | AP 0.5479 | F1 0.6799 | Time 0.07
Epoch 0004: Loss 13862.7412 | AUC 0.7565 | Recall 0.6455 | Precision 0.6455 | AP 0.6182 | F1 0.6886 | Time 0.08
Epoch 0005: Loss 19502.8555 | AUC 0.7600 | Recall 0.6449 | Precision 0.6449 | AP 0.6242 | F1 0.6884 | Time 0.09
Epoch 0006: Loss 18901.1074 | AUC 0.7600 | Recall 0.6449 | Precision 0.6449 | AP 0.6241 | F1 0.6884 | Time 0.08
Epoch 0007: Loss 14229.8262 | AUC 0.7564 | Recall 0.6447 | Precision 0.6447 | AP 0.6184 | F1 0.6883 | Time 0.09
Epoch 0008: Loss 9920.7012 | AUC 0.7438 | Recall 0.6521 | Precision 0.6521 | AP 0.6070 | F1 0.6937 | Time

### AnomalyDAE

In [29]:
def predict_anomalydae(label_test, anomalydae_compile, test_graph, test_node_features):

    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)
    
    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features

    anomalydae_ip_pred_res, anomalydae_ip_score_res = anomalydae_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    
    precision_pygod = eval_precision_at_k(label_test, anomalydae_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, anomalydae_ip_score_res)
    f1_score = 2*(precision_pygod*recall_pygod)/(precision_pygod+recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return  precision_pygod, recall_pygod, f1_score

def make_anomalydae_model(train_graph, train_node_features, 
                        label_train):

    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)
    
    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    
    anomalydae_model = AnomalyDAE(epoch=5, verbose=3, gpu=-1)
    anomalydae_compile = anomalydae_model.fit(pyG_train)

    return anomalydae_compile

#### Skenario 1

In [46]:
anomalydae_model = make_anomalydae_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_anomalydae(label_test, anomalydae_model, test_graph, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_14424\762223058.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)


Epoch 0000: Loss 1250.0575 |  | Time 8.44
Epoch 0001: Loss 155065.2656 |  | Time 8.61
Epoch 0002: Loss 61847.4414 |  | Time 7.53
Epoch 0003: Loss 8255.8564 |  | Time 7.63
Epoch 0004: Loss 101.1241 |  | Time 7.50


C:\Users\asus\AppData\Local\Temp\ipykernel_14424\762223058.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 0.1138 | AUC 0.8649 | Recall 0.7858 | Precision 0.7858 | AP 0.8266 | F1 0.7858 | Time 2.31
F1 score:  tensor(0.7858)
Precision:  tensor(0.7858)
Recall:  tensor(0.7858)


#### skenario 2

In [26]:
anomalydae_model = make_anomalydae_model(train_graph_2, train_node_features_2, label_train_2)
precision_score, recall_score, f1_score = predict_anomalydae(label_train_2, anomalydae_model, train_graph_2, train_node_features_2)

IndexError: index 15473 is out of bounds for dimension 0 with size 15473

#### skenario 3

In [30]:
anomalydae_model = make_anomalydae_model(train_graph_3, train_node_features_3, label_train_3)
precision_score, recall_score, f1_score = predict_anomalydae(label_test_3, anomalydae_model, test_graph_3, test_node_features_3)

IndexError: index 15473 is out of bounds for dimension 0 with size 15473

### CONAD

In [26]:
def predict_conad(label_test, conda_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features
    
    conad_ip_pred_res, conad_ip_score_res = conda_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    precision = eval_precision_at_k(label_test, conad_ip_score_res)
    recall = eval_recall_at_k(label_test, conad_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    return precision, recall, f1_score

def make_conad_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features

    conad_model = CONAD(epoch=5, gpu=-1, verbose=3)
    conad_compile = conad_model.fit(pyG_train)

    return conad_compile

#### Skenario 1

In [78]:
conad_model = make_conad_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_conad(label_test, conad_model, test_graph, test_node_features)

Epoch 0000: Loss 339767.2500 |  | Time 2.49
Epoch 0001: Loss 175259.7656 |  | Time 2.94
Epoch 0002: Loss 86086.2656 |  | Time 2.32
Epoch 0003: Loss 41946.2266 |  | Time 2.12
Epoch 0004: Loss 22432.9570 |  | Time 2.41
Epoch 0005: Loss 15288.9619 |  | Time 1.92
Epoch 0006: Loss 13692.7227 |  | Time 1.96
Epoch 0007: Loss 13934.9629 |  | Time 1.95
Epoch 0008: Loss 14214.0293 |  | Time 1.85
Epoch 0009: Loss 13812.2324 |  | Time 1.90
Epoch 0010: Loss 12621.3506 |  | Time 1.83
Epoch 0011: Loss 10911.8545 |  | Time 2.02
Epoch 0012: Loss 9074.7607 |  | Time 2.06
Epoch 0013: Loss 7414.6177 |  | Time 1.92
Epoch 0014: Loss 6072.0015 |  | Time 1.96
Epoch 0015: Loss 5046.2222 |  | Time 1.82
Epoch 0016: Loss 4261.4849 |  | Time 1.83
Epoch 0017: Loss 3630.8293 |  | Time 1.89
Epoch 0018: Loss 3094.5962 |  | Time 2.13
Epoch 0019: Loss 2629.7449 |  | Time 1.96
Epoch 0020: Loss 2238.3459 |  | Time 2.01
Epoch 0021: Loss 1928.6136 |  | Time 1.88
Epoch 0022: Loss 1700.3059 |  | Time 1.83
Epoch 0023: Loss 154

#### skenario 2

In [28]:
conad_model = make_conad_model(train_graph_2, train_node_features_2, label_train_2)
precision_score, recall_score, f1_score = predict_conad(label_train_2, conad_model, train_graph_2, train_node_features_2)

IndexError: The shape of the mask [15473] at index 0 does not match the shape of the indexed tensor [15480, 1] at index 0

#### skenario 3

In [27]:
conad_model = make_conad_model(train_graph_3, train_node_features_3, label_train_3)
precision_score, recall_score, f1_score = predict_conad(label_test_3, conad_model, train_graph_3, test_node_features_3)

IndexError: The shape of the mask [15473] at index 0 does not match the shape of the indexed tensor [15480, 1] at index 0

## Model with Hyperparameter

### DOMINANT

In [31]:
def predict_hdominant(label_test, dominant_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features
    dominant_ip_pred_res, dominant_ip_score_res = dominant_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True)
    
    precision_pygod = eval_precision_at_k(label_test, dominant_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, dominant_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return precision_pygod, recall_pygod, f1_score

def make_hdominant_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    dominant_model = DOMINANT(gpu=0, weight=1, weight_decay=3, num_layers=16, hid_dim=16, contamination=0.37, lr=0.001, verbose=3, epoch=5)
    dominant_compile = dominant_model.fit(pyG_train)
    return dominant_compile

In [32]:
dominant_model = make_hdominant_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_hdominant(label_test, dominant_model, test_graph, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_14424\3288320421.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)
C:\Users\asus\AppData\Local\Temp\ipykernel_14424\3288320421.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train)


Epoch 0000: Loss 75.1450 |  | Time 7.23
Epoch 0001: Loss 75.1340 |  | Time 0.75
Epoch 0002: Loss 75.1235 |  | Time 0.68
Epoch 0003: Loss 75.1135 |  | Time 0.73
Epoch 0004: Loss 75.1038 |  | Time 0.72


C:\Users\asus\AppData\Local\Temp\ipykernel_14424\3288320421.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 0.0098 | AUC 0.8024 | Recall 0.6911 | Precision 0.6911 | AP 0.7535 | F1 0.6911 | Time 0.18
F1 score:  tensor(0.6911)
Precision:  tensor(0.6911)
Recall:  tensor(0.6911)


### OCGNN

In [34]:
def predict_hocgnn(label_test, ocgnn_compile, pyG_test, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(pyG_test)
    pyG_test.x = test_node_features

    ocgnn_ip_pred_res, ocgnn_ip_score_res = ocgnn_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')

    precision_pygod = eval_precision_at_k(label_test, ocgnn_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, ocgnn_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    f1_pygod = eval_f1(label_test, ocgnn_ip_pred_res)
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    print("F1 pygod: ", f1_pygod)

    return precision_pygod, recall_pygod, f1_score

def make_hocgnn_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features

    ocgnn_model = OCGNN(hid_dim=10, num_layers=14, weight_decay=1, 
                    contamination=0.37, lr=0.001, epoch=5, gpu=-1, 
                    verbose=3)
    ocgnn_compile = ocgnn_model.fit(pyG_train)
    return ocgnn_compile

In [35]:
ocgnn_model = make_hocgnn_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_hocgnn(label_test, ocgnn_model, test_graph, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_14424\674703128.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)


Epoch 0000: Loss 53.9027 |  | Time 0.24
Epoch 0001: Loss 29.0920 |  | Time 0.24
Epoch 0002: Loss 24.1405 |  | Time 0.24
Epoch 0003: Loss 19.9970 |  | Time 0.27
Epoch 0004: Loss 16.7464 |  | Time 0.28


C:\Users\asus\AppData\Local\Temp\ipykernel_14424\674703128.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 0.0020 | AUC 0.3243 | Recall 0.2886 | Precision 0.2886 | AP 0.5252 | F1 0.2886 | Time 0.05
F1 score:  tensor(0.2886)
Precision:  tensor(0.2886)
Recall:  tensor(0.2886)
F1 pygod:  0.2951967173545739


### GAE

In [37]:
def predict_hgae(label_test, gae_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    gae_ip_pred_res, gae_ip_score_res = gae_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    precision = eval_precision_at_k(label_test, gae_ip_score_res)
    recall = eval_recall_at_k(label_test, gae_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return precision, recall, f1_score
    
def make_hgae_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train

    gae_model = GAE(hid_dim=12, num_layers=12, weight_decay=3,
                contamination=0.37, lr=0.001, epoch=5, gpu=-1,
                verbose=3, recon_s=True, sigmoid_s=True)
    gae_compile = gae_model.fit(pyG_train, label_train)
    return gae_compile

In [38]:
gae_model = make_hgae_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_hgae(label_test, gae_model, test_graph, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_14424\969071201.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)


Epoch 0000: Loss 0.4540 | AUC 0.9597 | Recall 0.8432 | Precision 0.8432 | AP 0.9354 | F1 0.8432 | Time 6.15
Epoch 0001: Loss 0.3589 | AUC 0.9597 | Recall 0.8432 | Precision 0.8432 | AP 0.9355 | F1 0.8432 | Time 4.28
Epoch 0002: Loss 0.2840 | AUC 0.9598 | Recall 0.8433 | Precision 0.8433 | AP 0.9356 | F1 0.8433 | Time 4.17
Epoch 0003: Loss 0.2510 | AUC 0.9603 | Recall 0.8443 | Precision 0.8443 | AP 0.9362 | F1 0.8443 | Time 3.88
Epoch 0004: Loss 0.2500 | AUC 0.4657 | Recall 0.2591 | Precision 0.2591 | AP 0.3375 | F1 0.2848 | Time 3.92


C:\Users\asus\AppData\Local\Temp\ipykernel_14424\969071201.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 0.0000 | AUC 0.4685 | Recall 0.8513 | Precision 0.8513 | AP 0.4407 | F1 0.0735 | Time 0.92
F1 score:  tensor(0.8513)
Precision:  tensor(0.8513)
Recall:  tensor(0.8513)


### AnomalyDAE

In [42]:
def predict_hanomalydae(label_test, anomalydae_compile, test_graph, test_node_features):

    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)
    
    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features

    anomalydae_ip_pred_res, anomalydae_ip_score_res = anomalydae_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    
    precision_pygod = eval_precision_at_k(label_test, anomalydae_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, anomalydae_ip_score_res)
    f1_score = 2*(precision_pygod*recall_pygod)/(precision_pygod+recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return  precision_pygod, recall_pygod, f1_score

def make_hanomalydae_model(train_graph, train_node_features, 
                        label_train):

    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)
    
    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    
    anomalydae_model = AnomalyDAE(hid_dim=12, emb_dim=4, 
                        lr=0.001, contamination=0.37,
                        epoch=5, gpu=0,
                        weight=1, verbose=3)
    anomalydae_compile = anomalydae_model.fit(pyG_train)

    return anomalydae_compile

In [43]:
anomalydae_model = make_hanomalydae_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_hanomalydae(label_test, anomalydae_model, test_graph, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_14424\1419949797.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)


Epoch 0000: Loss 11186.1670 |  | Time 0.64
Epoch 0001: Loss 114.6346 |  | Time 0.75
Epoch 0002: Loss 114.4019 |  | Time 0.78
Epoch 0003: Loss 114.1781 |  | Time 0.75
Epoch 0004: Loss 113.9597 |  | Time 0.75


C:\Users\asus\AppData\Local\Temp\ipykernel_14424\1419949797.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 0.0239 | AUC 0.8121 | Recall 0.7875 | Precision 0.7875 | AP 0.7600 | F1 0.7875 | Time 0.14
F1 score:  tensor(0.7875)
Precision:  tensor(0.7875)
Recall:  tensor(0.7875)


### CONAD

In [40]:
def predict_hconad(label_test, conda_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features
    
    conad_ip_pred_res, conad_ip_score_res = conda_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    precision = eval_precision_at_k(label_test, conad_ip_score_res)
    recall = eval_recall_at_k(label_test, conad_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    return precision, recall, f1_score

def make_hconad_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features

    conad_model = CONAD(hid_dim=10, num_layers=16, 
                        lr=0.001, weight_decay= 1, contamination=0.37,
                        epoch=5, gpu=-1,  
                        weight=1, dropout=0.2, verbose=3)
    conad_compile = conad_model.fit(pyG_train)

    return conad_compile

In [41]:
conad_model = make_hconad_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_hconad(label_test, conad_model, test_graph, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_14424\1762596898.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)


Epoch 0000: Loss 37.6440 |  | Time 4.33
Epoch 0001: Loss 37.6382 |  | Time 4.45
Epoch 0002: Loss 37.6337 |  | Time 4.22
Epoch 0003: Loss 37.6333 |  | Time 4.57
Epoch 0004: Loss 37.6260 |  | Time 4.22


C:\Users\asus\AppData\Local\Temp\ipykernel_14424\1762596898.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 0.0098 | AUC 0.8027 | Recall 0.6913 | Precision 0.6913 | AP 0.7539 | F1 0.6913 | Time 0.57
F1 score:  tensor(0.6913)
Precision:  tensor(0.6913)
Recall:  tensor(0.6913)
